In [30]:
import os
import time
import argparse
import networkx as nx
from tqdm import tqdm
from itertools import chain
from kg_connection import ASERKGConnection

In [31]:
kg_path = "/home/data/corpora/aser/database/"
networkx_path = "/home/data/jchengaj/aser_data/"

# kg_fn = 'filter_2.0/2/KG.db'
# to_fn = 'test.pickle'
# kg_fn = 'filter_2.0/100/KG.db'
# to_fn = 'core_100.pickle'
# kg_fn = 'filter_2.0/50/KG.db'
# to_fn = 'core_50.pickle'
# kg_fn = 'filter_2.0/10/KG.db'
# to_fn = 'core_10.pickle'
kg_fn = 'filter_2.0/5/KG.db'
to_fn = 'core_5.pickle'

In [ ]:
# set input & output paths
kg_path = os.path.join(kg_path, kg_fn)
networkx_path = os.path.join(networkx_path, to_fn)

In [32]:
kg_conn = ASERKGConnection(kg_path,
                           mode='memory', 
                           grain="words", 
                           load_types=["merged_eventuality", "words", "eventuality"])

13766746it [14:18, 16030.53it/s]
30309830it [12:33, 40244.20it/s]


In [33]:
G_aser = nx.DiGraph()

In [34]:
# temporarily used for getting event info from ASER connection
def get_eventuality_info(kg_conn, strs):
    eid_list = kg_conn.merged_eventuality_cache.get(strs, [])
    event_list = [kg_conn.eid2eventuality_cache[eid] for eid in eid_list]

    info_list = list()
    for e in event_list:
        info = str(e)
        info_list.append(info)
    info_list = list(set(info_list))
    return info_list

In [ ]:
for node in tqdm(kg_conn.merged_eventuality_cache):# the cache is a map from e_words(str) to e_id
    G_aser.add_node(node,
                    freq=kg_conn.get_event_frequency(node),
                    info=get_eventuality_info(kg_conn, node))

 50%|███████████████               | 6055024/12010922 [30:03<27:21, 3628.00it/s]

In [ ]:
# traverse all the nodes, and get it's neighbors
gather_relations = lambda key, successor_dict: \
    list(set(chain(*[list(item[1].keys()) for item in successor_dict[key]])))
gather_weights = lambda key, successor_dict, rels: \
    dict([(r, sum([item[1].get(r, 0) for item in successor_dict[key]])) for r in rels])

In [ ]:
for node in tqdm(kg_conn.merged_eventuality_cache):
    successor_dict = kg_conn.merged_eventuality_relation_cache["head_words"].get(node, {})
    selected_tails = [(key, gather_weights(key, successor_dict, gather_relations(key, successor_dict))) \
                      for key, relations in successor_dict.items()]  # tail, relation
    for key, rels in selected_tails:
        G_aser.add_edge(node, key,
                        relations=rels, )

In [ ]:
nx.write_gpickle(G_aser, networkx_path)